# Processamento de Feriados do Brasil (2024-2025)

Este notebook baixa, padroniza e salva os feriados nacionais, estaduais, municipais e facultativos do Brasil.

## 1. Importação de Bibliotecas

In [7]:
%pip install requests


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
import requests
import pandas as pd
from io import StringIO

## 2. Configuração de URLs e Parâmetros

In [9]:
# URLs dos arquivos CSV de feriados para 2024 e 2025
base_urls = {
    'nacional': 'https://github.com/joaopbini/feriados-brasil/raw/master/dados/feriados/nacional/csv/',
    'estadual': 'https://github.com/joaopbini/feriados-brasil/raw/master/dados/feriados/estadual/csv/',
    'municipal': 'https://github.com/joaopbini/feriados-brasil/raw/master/dados/feriados/municipal/csv/',
    'facultativo': 'https://github.com/joaopbini/feriados-brasil/raw/master/dados/feriados/facultativo/csv/'
}
anos = ['2024', '2025']

# Definir colunas padrão
colunas_finais = ['Data', 'Nome_Feriado', 'Tipo_Feriado', 'Descricao', 'Sigla_Estado', 'Municipio']

## 3. Criação de Diretórios

In [10]:
# Pasta de destino
pasta_destino = "../../data/processed/"
pasta_feriados = os.path.join(pasta_destino, 'feriados')
os.makedirs(pasta_feriados, exist_ok=True)

# Lista para armazenar todos os DataFrames
todos_dataframes = []

## 4. Download e Processamento dos Dados

In [11]:
for categoria, base_url in base_urls.items():
    for ano in anos:
        url = f'{base_url}{ano}.csv'
        print(f'Baixando {url} ...')
        try:
            resp = requests.get(url)
            resp.raise_for_status()
            
            df = pd.read_csv(StringIO(resp.text))
            df.columns = colunas_finais[:len(df.columns)]
            
            # Garantir que todas as colunas finais existam
            for coluna in colunas_finais:
                if coluna not in df.columns:
                    df[coluna] = ''
            
            df = df[colunas_finais]
            
            # Filtrar apenas registros de SP ou NACIONAL
            df_filtrado = df[(df['Sigla_Estado'] == 'SP') | (df['Tipo_Feriado'] == 'NACIONAL')]
            
            # Criar diretório do ano se não existir
            pasta_ano = os.path.join(pasta_feriados, ano)
            os.makedirs(pasta_ano, exist_ok=True)
            
            # Montar nome de arquivo e salvar
            nome_arquivo = f'{categoria}_{ano}.csv'
            caminho_arquivo = os.path.join(pasta_ano, nome_arquivo)
            df.to_csv(caminho_arquivo, index=False, encoding='utf-8')

            # Armazenar DataFrame filtrado na lista geral
            todos_dataframes.append(df_filtrado)
            print(f'Dados de {categoria} {ano} salvos em {caminho_arquivo} ({len(df)} registros)')
            print(f'Registros filtrados (SP/NACIONAL): {len(df_filtrado)}')
                        
        except Exception as e:
            print(f'Erro ao baixar {url}: {e}')

Baixando https://github.com/joaopbini/feriados-brasil/raw/master/dados/feriados/nacional/csv/2024.csv ...
Dados de nacional 2024 salvos em ../../data/processed/feriados/2024/nacional_2024.csv (8 registros)
Registros filtrados (SP/NACIONAL): 8
Baixando https://github.com/joaopbini/feriados-brasil/raw/master/dados/feriados/nacional/csv/2025.csv ...
Dados de nacional 2025 salvos em ../../data/processed/feriados/2025/nacional_2025.csv (8 registros)
Registros filtrados (SP/NACIONAL): 8
Baixando https://github.com/joaopbini/feriados-brasil/raw/master/dados/feriados/estadual/csv/2024.csv ...
Dados de estadual 2024 salvos em ../../data/processed/feriados/2024/estadual_2024.csv (48 registros)
Registros filtrados (SP/NACIONAL): 1
Baixando https://github.com/joaopbini/feriados-brasil/raw/master/dados/feriados/estadual/csv/2025.csv ...
Dados de estadual 2025 salvos em ../../data/processed/feriados/2025/estadual_2025.csv (48 registros)
Registros filtrados (SP/NACIONAL): 1
Baixando https://github.co

## 5. Consolidação dos Dados

In [12]:
# Juntar todos os DataFrames em um único arquivo
if todos_dataframes:
    print(f'Combinando {len(todos_dataframes)} DataFrames...')
    df_combinado = pd.concat(todos_dataframes, ignore_index=True)
    
    # Salvar arquivo consolidado
    arquivo_final = os.path.join(pasta_feriados, 'feriados_completo.csv')
    df_combinado.to_csv(arquivo_final, index=False)
    print(f'Arquivo consolidado salvo em: {arquivo_final} ({len(df_combinado)} registros)')
else:
    print('Nenhum dado foi carregado')

Combinando 8 DataFrames...
Arquivo consolidado salvo em: ../../data/processed/feriados/feriados_completo.csv (2719 registros)
